In [83]:
# queremos aquí definir una u-net para solucionar un problema de in-painting
# la entrada será una imagen de 44x2664 y la salida también
# la salida será la imagen original con filas de ceros en las filas que queremos rellenar

# la idea es que la red aprenda a rellenar las filas de ceros de la imagen de entrada

# la red tendrá una arquitectura u-net con las siguientes características:
# - capa de entrada
# - 4 bloques convolucionales con maxpooling y dropout
# - capa de bottleneck
# - 4 bloques de convolución con upsampling y dropout
# - capa de salida

# la función de activación será ReLu en todas las capas incluída la de salida porque tenemos que predecir valores entre 0 y más de 1

# la función de pérdida será el error cuadrático medio porque es un problema de regresión
# la métrica será el error absoluto medio porque es más interpretable

# la red se entrenará con el optimizador Adam con una tasa de aprendizaje de 0.001

# la red se entrenará durante 1000 épocas con un tamaño de batch de 5

# la red se guardará en un fichero llamado 'modelo_inpainting.h5'

# importamos las librerías necesarias

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Lambda
from tensorflow.keras.layers import LeakyReLU  # Cambiada esta línea
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import backend as K

# # definimos la función de pérdida y la métrica

# def mean_squared_error(y_true, y_pred):
#     return K.mean(K.square(y_true - y_pred))

# def mean_absolute_error(y_true, y_pred):
#     return K.mean(K.abs(y_true - y_pred))

# definimos la arquitectura de la red

def unet_inpainting(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):

    # importar inicializador de semilla
    from tensorflow.keras.initializers import RandomNormal, RandomUniform, TruncatedNormal, VarianceScaling, glorot_normal, glorot_uniform, he_normal, he_uniform

    # Importar funciones de activación
    from tensorflow.keras.layers import LeakyReLU, PReLU, ReLU, ELU, ThresholdedReLU

    # Definir inicializador con semilla
    initializer = glorot_normal(seed=0)

    # Definir función de activación
    activation = LeakyReLU(alpha=0.3)



    
    kernel_size = (3, 3)
    # Capa de entrada
    inputs = Input(input_shape)

    # Bloque 1
    conv1 = Conv2D(32, (1500,6), activation=activation, padding='same', kernel_initializer=initializer)(inputs)
    conv1 = Conv2D(32, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # Bloque 2
    conv2 = Conv2D(64, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(pool1)
    conv2 = Conv2D(64, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 1))(conv2)

    # Bloque 3
    conv3 = Conv2D(128, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(pool2)
    conv3 = Conv2D(128, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 1))(conv3)

    # Bloque 4
    conv4 = Conv2D(256, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(pool3)
    conv4 = Conv2D(256, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(conv4)
    drop4 = Dropout(0)(conv4)
    pool4 = MaxPooling2D(pool_size=(3, 2))(drop4)

    # Bloque 5
    conv41 = Conv2D(512, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(pool4)
    conv41 = Conv2D(512, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(conv41)
    drop41 = Dropout(0.01)(conv41)
    pool41 = MaxPooling2D(pool_size=(3, 1))(drop41)



    # Capa de bottleneck
    conv5 = Conv2D(1024, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(pool41)
    conv5 = Conv2D(1024, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(conv5)

    # Bloque 61
    up61 = Conv2DTranspose(512, kernel_size, strides=(3, 1), padding='same', kernel_initializer=initializer)(conv5)
    up61 = concatenate([up61, drop41], axis=3)
    conv61 = Conv2D(512, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(up61)
    conv61 = Conv2D(512, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(conv61)

    # Bloque 6
    up6 = Conv2DTranspose(256, kernel_size, strides=(3, 2), padding='same', kernel_initializer=initializer)(conv61)
    up6 = concatenate([up6, drop4], axis=3)
    conv6 = Conv2D(256, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(up6)
    conv6 = Conv2D(256, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(conv6)

    # Bloque 7
    up7 = Conv2DTranspose(128, kernel_size, strides=(2, 1), padding='same', kernel_initializer=initializer)(conv6)
    up7 = concatenate([up7, conv3], axis=3)
    conv7 = Conv2D(128, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(up7)
    conv7 = Conv2D(128, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(conv7)

    # Bloque 8
    up8 = Conv2DTranspose(64, kernel_size, strides=(2, 1), padding='same', kernel_initializer=initializer)(conv7)
    up8 = concatenate([up8, conv2], axis=3)
    conv8 = Conv2D(64, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(up8)
    conv8 = Conv2D(64, kernel_size, activation=activation, padding='same', kernel_initializer=initializer)(conv8)

    # Bloque 9
    up9 = Conv2DTranspose(32, kernel_size, strides=(2, 2), padding='same', kernel_initializer=initializer)(conv8)
    up9 = concatenate([up9, conv1, inputs], axis=3)
    conv9 = Conv2D(16, (1,1), activation=activation, padding='same', kernel_initializer=initializer)(up9)
    conv10 = Conv2D(16, (1,1), activation=activation, padding='same', kernel_initializer=initializer)(conv9)
    conv11 = Conv2D(1, (1,1), padding='same', activation=activation, kernel_initializer=initializer)(conv10)
    

    # # añadir un bloque más para poder tener la imagen de entrada disponible en la capa de salida. Porque hay datos en la salida que tienen que ser exactamente los de la entrada

    # up10 = concatenate([conv9, inputs], axis=3)

    # # Capa de salida
    # outputs = Conv2D(1, 1, activation='relu')(up10)

    # Crear el modelo
    model = Model(inputs=inputs, outputs=conv11)

    return model


# obtener el índice de ejecución actual
cell_index=(get_ipython().execution_count)